## Unit testing the EWR tool

In [190]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [191]:
import unittest
import pandas as pd
from pandas._testing import assert_frame_equal
from datetime import datetime

import numpy as np

import dashboard, data_inputs, gauge_getter, scenario_handling, observed_handling, evaluate_EWRs, summarise_results

#### Testing functions in data_inputs

In [193]:
class test_data_inputs(unittest.TestCase):
    def test_convert_max_interevent(self):
        unique_water_years = [2012, 2013, 2014, 2015]
        water_years = [2012]*365+[2013]*365+[2014]*365+[2015]*365
        EWR_info = {'max_inter-event': 1}
        new_max_interevent = data_inputs.convert_max_interevent(unique_water_years, water_years, EWR_info)
        self.assertEqual(new_max_interevent, 365)
    def test_gauge_to_catchment(self):
        expected_catchment = 'Gwydir'
        input_gauge = '418026'
        catchment = data_inputs.gauge_to_catchment(input_gauge)
        self.assertEqual(catchment, expected_catchment)
        expected_catchment = 'Namoi'
        input_gauge = '419049'
        catchment = data_inputs.gauge_to_catchment(input_gauge)
        self.assertEqual(catchment, expected_catchment)
    def test_wy_to_climate(self):
        data = {'418026': list(range(0,730,1)), '425012': list(range(0,1460,2))}
        index = pd.date_range(start='1/1/2017', end='31/12/2018')
        df = pd.DataFrame(data = data, index = index)
        climate_daily = data_inputs.wy_to_climate(df, 'Gwydir', 'Standard - 1911 to 2018 climate categorisation')
        expected_climate_daily = np.array(['Dry']*365 + ['Very Dry']*365)
        self.assertTrue(np.array_equal(climate_daily, expected_climate_daily))
    def test_get_multi_gauges(self):
        # Testing the return all request
        expected_multi_gauges = {'PU_0000130': {'421090': '421088', '421088': '421090'},
                                 'PU_0000131': {'421090': '421088', '421088': '421090'},
                                 'PU_0000132': {'421090': '421088', '421088': '421090'},
                                 'PU_0000133': {'421090': '421088', '421088': '421090'}}
        multi_gauges = data_inputs.get_multi_gauges('all')
        self.assertDictEqual(expected_multi_gauges, multi_gauges)
        # Testing the return just gauges request
        expected_multi_gauges = {'421090': '421088', '421088': '421090'}
        multi_gauges = data_inputs.get_multi_gauges('gauges')
        self.assertDictEqual(expected_multi_gauges, multi_gauges)
    def test_get_simultaneous_gauges(self):
        # Testing the return all request
        expected_sim_gauges = {'PU_0000131': {'421090': '421022', '421022': '421090'},
                               'PU_0000132': {'421090': '421022', '421022': '421090'},
                               'PU_0000133': {'421090': '421022', '421022': '421090'}}
        sim_gauges = data_inputs.get_simultaneous_gauges('all')
        self.assertDictEqual(expected_sim_gauges, sim_gauges)
        # Testing the return just gauges request
        expected_sim_gauges = {'421090': '421022', '421022': '421090'}
        sim_gauges = data_inputs.get_simultaneous_gauges('gauges')
        self.assertDictEqual(expected_sim_gauges, sim_gauges)        
        

In [194]:
Test = test_data_inputs()
suite = unittest.TestSuite()

suite.addTest(test_data_inputs('test_convert_max_interevent'))
suite.addTest(test_data_inputs('test_gauge_to_catchment'))
suite.addTest(test_data_inputs('test_wy_to_climate'))
suite.addTest(test_data_inputs('test_get_multi_gauges'))
suite.addTest(test_data_inputs('test_get_simultaneous_gauges'))

unittest.TextTestRunner().run(suite)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.061s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Testing functions in evaluate EWRs

In [185]:
class test_evaluate_EWRs(unittest.TestCase):
    def test_component_pull(self):
        EWR_table, seeNotesEwrs, undefinedEwrs, noThresh_df, noDurationEwrs, DSFewrs = data_inputs.get_EWR_table()
        gauge = '409025'
        PU = 'PU_0000253'
        EWR = 'SF1_P'
        component = 'duration'
        self.assertEqual(evaluate_EWRs.component_pull(EWR_table, gauge, PU, EWR, component), '10')
        
    def test_apply_correction(self):
        info = 960
        correction = 1.2
        self.assertEqual(evaluate_EWRs.apply_correction(info, correction), 1152.0)
        
    def test_get_EWRs(self):
        EWR_table, seeNotesEwrs, undefinedEwrs, noThresh_df, noDurationEwrs, DSFewrs = data_inputs.get_EWR_table()
        PU = 'PU_0000283'
        gauge = '410007'
        EWR = 'SF1_P'
        minThreshold_tolerance = (100 - 0)/100
        maxThreshold_tolerance = (100 + 0)/100
        duration_tolerance = (100 - 0)/100
        drawdown_tolerance = (100 - 0)/100
        allowance ={'minThreshold': minThreshold_tolerance, 'maxThreshold': maxThreshold_tolerance,
                        'duration': duration_tolerance, 'drawdown': drawdown_tolerance}
        components = ['SM', 'EM']
        
        expected = {'gauge': '410007', 'planning_unit': 'PU_0000283', 'EWR_code': 'SF1_P', 'start_day': None, 'start_month': 10, 'end_day': None, 'end_month':4}
        self.assertEqual(evaluate_EWRs.get_EWRs(PU, gauge, EWR, EWR_table, allowance, components), expected)
    
    def test_mask_dates(self):
        ''' This testing function will also be testing the functions get_month_mask, get_day_month_mask, and get_day_month_mask in the same module'''
        #------------ Dataframe to be passed to all testing functions here ----------#
        
        data = {'409102': list(range(0,3650,10)), '425012': list(range(0,3650,10))}
        index = pd.date_range(start='1/1/2019', end='31/12/2019')
        df = pd.DataFrame(data = data, index = index)
        
        #----------------------------------------------------------------------------#
        # Testing for no filtering (all year round EWR requirement)
        EWR_info = {'start_day': None, 'end_day': None, 'start_month': 7, 'end_month': 6}
        masked_7to6 = set(pd.date_range(start='1/1/2019', end='31/12/2019'))
        self.assertEqual(evaluate_EWRs.mask_dates(EWR_info, df), masked_7to6)
        # Testing for a month subset
        EWR_info = {'start_day': None, 'end_day': None, 'start_month': 7, 'end_month': 9}
        masked_7to9 = set(pd.date_range(start='2019-07-01', end='2019-09-30'))
        self.assertEqual(evaluate_EWRs.mask_dates(EWR_info, df), masked_7to9)
        # Testing for water year crossover EWR requirement:
        EWR_info = {'start_day': None, 'end_day': None, 'start_month': 6, 'end_month': 8}
        masked_6to8 = set(pd.date_range(start='2019-06-01', end='2019-08-31'))
        self.assertEqual(evaluate_EWRs.mask_dates(EWR_info, df), masked_6to8)
        # Testing for start day and end day within same month inclusion in the EWR requirement:
        EWR_info = {'start_day': 12, 'end_day': 28, 'start_month': 6, 'end_month': 6}
        masked_612to628 = set(pd.date_range(start='2019-06-12', end='2019-06-28'))
        self.assertEqual(evaluate_EWRs.mask_dates(EWR_info, df), masked_612to628) 
        # Testing for start day and end day within different months inclusion in the EWR requirement:
        EWR_info = {'start_day': 12, 'end_day': 18, 'start_month': 8, 'end_month': 9}
        masked_812to918 = set(pd.date_range(start='2019-08-12', end='2019-09-18'))
        self.assertEqual(evaluate_EWRs.mask_dates(EWR_info, df), masked_812to918)
    def test_wateryear_daily(self):
        # Testing non standard water year part of function:
        EWR_info = {'start_day': None, 'end_day': None, 'start_month': 5, 'end_month': 8}
        data = {'409102': list(range(0,3650,10)), '425012': list(range(0,3650,10))}
        index = pd.date_range(start='1/1/2019', end='31/12/2019')
        df = pd.DataFrame(data = data, index = index)
        expected_2018 = [2018]*120
        expected_2019 = [2019]*245
        expected_array = np.array(expected_2018 + expected_2019)
        array = evaluate_EWRs.wateryear_daily(df, EWR_info)
        self.assertTrue(np.array_equal(array, expected_array))
        # Testing standard water year part of function:
        EWR_info = {'start_day': None, 'end_day': None, 'start_month': 7, 'end_month': 9}
        data = {'409102': list(range(0,3650,10)), '425012': list(range(0,3650,10))}
        index = pd.date_range(start='1/1/2019', end='31/12/2019')
        df = pd.DataFrame(data = data, index = index)
        expected_2018 = [2018]*181
        expected_2019 = [2019]*184
        expected_array = np.array(expected_2018 + expected_2019)
        array = evaluate_EWRs.wateryear_daily(df, EWR_info)
        self.assertTrue(np.array_equal(array, expected_array))
    def test_which_wateryear(self):
        # Testing for equal event for each water year:
        i = 45
        event = [10]*10
        water_years = np.array([2018]*40+[2019]*10)
        water_year = evaluate_EWRs.which_water_year(i, event, water_years)
        self.assertEqual(water_year, 2019)
        # Testing for non-equal event for each water year:
        i = 42
        event = [10]*9
        water_years = np.array([2018]*40+[2019]*10)
        water_year = evaluate_EWRs.which_water_year(i, event, water_years)
        self.assertEqual(water_year, 2018)
    def test_get_duration(self):
        EWR_info = {'duration_VD': 2, 'duration': 5}
        duration = evaluate_EWRs.get_duration('Very Dry', EWR_info)
        self.assertEqual(duration, 2)
        EWR_info = {'duration_VD': None, 'duration': 5}
        duration = evaluate_EWRs.get_duration('Very Dry', EWR_info)
        self.assertEqual(duration, 5)
    def test_construct_event_dict(self):
        water_years = [2018]*365+[2019]*365
        all_events = evaluate_EWRs.construct_event_dict(water_years)
        expected_result = {2018:[], 2019:[]}
        self.assertEqual(all_events, expected_result)
    def test_get_days_between(self):
        # Testing low flow with more than 1 of interevent requirement:
        no_events = {2012: [np.array(735), np.array(50), np.array(2)], 2013: [np.array(35), np.array(50), np.array(365)],
                     2014: [np.array(35), np.array(50), np.array(2)], 2015: [np.array(35), np.array(280), np.array(2)]}
        years_with_events = [0,0,0,1] # This will be used in the calculation
        EWR = 'BF'
        EWR_info = {'max_inter-event': 2}
        unique_water_years = [2012, 2013, 2014, 2015]
        water_years = [2012]*365+[2013]*365+[2014]*365+[2015]*365
        days_between = evaluate_EWRs.get_days_between(years_with_events, no_events, EWR, EWR_info, unique_water_years, water_years)
        expected_days_between = [np.array([]), np.array([]), np.array([]), np.array([1095])]
        for i, v in enumerate(days_between):
            self.assertTrue(np.array_equal(days_between[i], expected_days_between[i]))
        # Testing low flow with less than 1 year of interevent requirement
        no_events = {2012: [np.array([35]), np.array([50]), np.array([2])], 2013: [np.array([35]), np.array([50]), np.array([2])],
                     2014: [np.array([35]), np.array([50]), np.array([2])], 2015: [np.array([35]), np.array([50]), np.array([2])]}
        years_with_events = [0,0,0,1] #This will be ignored in this calculation
        EWR = 'BF'
        EWR_info = {'max_inter-event': 0.04} # Equates to about 14-15 days
        unique_water_years = [2012, 2013, 2014, 2015]
        water_years = [2012]*365+[2013]*365+[2014]*365+[2015]*365
        days_between = evaluate_EWRs.get_days_between(years_with_events, no_events, EWR, EWR_info, unique_water_years, water_years)
        expected_days_between = [np.array([35, 50]), np.array([35, 50]), np.array([35, 50]), np.array([35, 50])]
        for i, v in enumerate(days_between):
            self.assertTrue(np.array_equal(days_between[i], expected_days_between[i]))
        # Testing non low flow EWR with more than 1 year requirement:
        no_events = {2012: [np.array([35]), np.array([50]), np.array([2])], 2013: [np.array([35]), np.array([50]), np.array([2])],
                     2014: [np.array([35]), np.array([50]), np.array([2])], 2015: [np.array([735]), np.array([2])]}
        years_with_events = [0,1,0,0] #This will be ignored in this calculation
        EWR = 'LF'
        EWR_info = {'max_inter-event': 2}
        unique_water_years = [2012, 2013, 2014, 2015]
        water_years = [2012]*365+[2013]*365+[2014]*365+[2015]*365
        days_between = evaluate_EWRs.get_days_between(years_with_events, no_events, EWR, EWR_info, unique_water_years, water_years)
        expected_days_between = [np.array([]), np.array([]), np.array([]), np.array([735])]
        for i, v in enumerate(days_between):
            self.assertTrue(np.array_equal(days_between[i], expected_days_between[i]))
        # Testing for EWR with only a subset of the water year available
        no_events = {2012: [np.array([35]), np.array([122]), np.array([2])], 2013: [np.array([35]), np.array([50]), np.array([2])],
                     2014: [np.array([35]), np.array([50]), np.array([2])], 2015: [np.array([730]), np.array([50]), np.array([121])]}
        years_with_events = [0,0,0,1] #This will be ignored in this calculation
        EWR = 'LF'
        EWR_info = {'max_inter-event': 2}
        unique_water_years = [2012, 2013, 2014, 2015]
        water_years = [2012]*365+[2013]*365+[2014]*365+[2015]*365
        days_between = evaluate_EWRs.get_days_between(years_with_events, no_events, EWR, EWR_info, unique_water_years, water_years)
        expected_days_between = [np.array([]), np.array([]), np.array([]), np.array([730])]
        for i, v in enumerate(days_between):          
            self.assertTrue(np.array_equal(days_between[i], expected_days_between[i]))
    def test_get_event_years(self):
        EWR_info = {'events_per_year': 2}
        events = {2012: [np.array([5]*5), np.array([10]*5), np.array([20*8])], 2013: [np.array([50]*20)],
                  2014: [np.array([5]*5), np.array([10]*5), np.array([20*8]), np.array([20*8])], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        durations = [5,5,5,5]
        event_years = evaluate_EWRs.get_event_years(EWR_info, events, unique_water_years, durations)
        expected_event_years = [1,0,1,0]
        self.assertEqual(event_years, expected_event_years)
        
        
    def test_get_achievements(self):
        # Testing 1 event per year requirement:
        EWR_info = {'events_per_year': 1}
        events = {2012: [np.array([5]*5), np.array([10]*5), np.array([20]*8)], 2013: [np.array([50]*20)],
                  2014: [np.array([5]*5), np.array([10]*5), np.array([20]*8), np.array([20]*8)], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        durations = [5,5,5,5]
        num_events = evaluate_EWRs.get_achievements(EWR_info, events, unique_water_years, durations)
        expected_num_events = [3,1,4,0]
        self.assertEqual(num_events, expected_num_events)
        # Testing 2 events per year requirement:
        EWR_info = {'events_per_year': 2}
        events = {2012: [np.array([5]*5), np.array([10]*5), np.array([20]*8)], 2013: [np.array([50]*20)],
                  2014: [np.array([5]*5), np.array([10]*5), np.array([20]*8), np.array([20]*8)], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        durations = [5,5,5,5]
        num_events = evaluate_EWRs.get_achievements(EWR_info, events, unique_water_years, durations)
        expected_num_events = [1,0,2,0]
        self.assertEqual(num_events, expected_num_events)
        
        
    def test_get_number_events(self):
        # Testing 1 event per year requirement:
        EWR_info = {'events_per_year': 1}
        events = {2012: [np.array([5]*5), np.array([10]*5), np.array([20]*8)], 2013: [np.array([50]*20)],
                  2014: [np.array([5]*5), np.array([10]*5), np.array([20]*8), np.array([20]*8)], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        durations = [5,5,5,5]
        num_events = evaluate_EWRs.get_number_events(EWR_info, events, unique_water_years, durations)
        expected_num_events = [3,1,4,0]
        self.assertEqual(num_events, expected_num_events)
        # Testing 2 events per year requirement:
        EWR_info = {'events_per_year': 2}
        events = {2012: [np.array([5]*5), np.array([10]*5), np.array([20]*8)], 2013: [np.array([50]*20)],
                  2014: [np.array([5]*5), np.array([10]*5), np.array([20]*8), np.array([20]*8)], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        durations = [5,5,5,5]
        num_events = evaluate_EWRs.get_number_events(EWR_info, events, unique_water_years, durations)
        expected_num_events = [3,1,4,0]
        self.assertEqual(num_events, expected_num_events)
        
        
    def test_get_average_event_length(self):
        events = {2012: [np.array([5]*5), np.array([10]*5), np.array([20]*8)], 2013: [np.array([50]*20)],
                  2014: [np.array([5]*5), np.array([10]*5), np.array([20]*8), np.array([20]*8)], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        average_length = evaluate_EWRs.get_average_event_length(events, unique_water_years)
        expected_average_length = [6,20,6.5,0]
        self.assertEqual(average_length, expected_average_length)
    def test_get_total_days(self):
        events = {2012: [np.array([5]*5), np.array([10]*5), np.array([20]*8)], 2013: [np.array([50]*20)],
                  2014: [np.array([5]*5), np.array([10]*5), np.array([20]*8), np.array([20]*8)], 2015: []}
        unique_water_years = [2012, 2013, 2014, 2015]
        total_days = evaluate_EWRs.get_total_days(events, unique_water_years)
        expected_total_days = [18,20,26,0]
        self.assertEqual(total_days, expected_total_days)
    def test_get_data_gap(self):
        data = {'409102': list(range(0,3650,10)), '425012': list(range(0,3650,10))}
        index = pd.date_range(start='1/1/2019', end='31/12/2019')
        df = pd.DataFrame(data = data, index = index)
        df.iloc[0:4] = None
        df.iloc[57] = np.nan
        unique_water_years = [2018]*181+[2019]*184
        gauge = '409102'
        missing_list = evaluate_EWRs.get_data_gap(df, unique_water_years, gauge)
        expected_missing = [5,0]
        self.assertEqual(missing_list, expected_missing)
    def test_check_flow(self):
        '''Things to check on the check functions:
        - Event requirements just triggered
        - Event requirements failed
        - event requirements failed but there is some gap track remaining
        - flow failed but event recorded
        '''
        # Set up variables for all tests:
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'gap_tolerance': 0, 'min_event':10}
        iteration = 50
        flow = 5
        event = [5]*9
        all_events = {2012:[np.array([10]*10), np.array([15]*12)], 2013:[np.array([10]*50)], 
                      2014:[np.array([10]*10), np.array([15]*15), np.array([10]*20)], 2015:[]}
        no_event = 50
        all_no_events = {2012:[np.array([25]), np.array([2])], 2013:[np.array([250])],
                         2014:[np.array([400]), np.array([2]), np.array([25])], 2015:[np.array([450])]}
        gap_track = 0
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*365)
        total_event = 9

        event, all_events, no_event, all_no_events, gap_track, total_event = evaluate_EWRs.flow_check(EWR_info, iteration, flow, event, all_events, no_event, all_no_events, gap_track, 
                       water_years, total_event)
        
        # Expected results - TEST 1: #
        expected_event = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
        expected_all_events = {2012:[np.array([10]*10), np.array([15]*12)], 2013:[np.array([10]*50)], 
                               2014:[np.array([10]*10), np.array([15]*15), np.array([10]*20)], 2015:[]}
        expected_no_event = 51
        expected_all_no_events = {2012:[np.array([25]), np.array([2])], 2013:[np.array([250])],
                                  2014:[np.array([400]), np.array([2]), np.array([25])], 2015:[np.array([450])]}
        expected_gap_track = 0
        expected_total_event = 10
        self.assertEqual(event,expected_event)
        for year in all_events:
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
        self.assertEqual(no_event, expected_no_event)
        self.assertDictEqual(all_no_events, expected_all_no_events)
        for year in all_no_events:
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(gap_track, expected_gap_track)
        self.assertEqual(total_event, expected_total_event)
        
    def test_lowflow_check(self):
        '''Things to check on the check functions:
        - Event requirements just triggered
        - Event requirements failed
        - event requirements failed but there is some gap track remaining
        - flow failed but event recorded
        '''
        # Set up variables for all tests:
        EWR_info = {'min_flow': 5, 'max_flow': 20}
        flow = 5
        water_year = 2015
        event = [5]*9
        iteration = 365+365+365+100
        all_events = {2012:[np.array([10]*10), np.array([15]*12)], 2013:[np.array([10]*50)], 
                      2014:[np.array([10]*10), np.array([15]*15), np.array([10]*20)], 2015:[]}
        no_event = 0
        all_no_events = {2012:[np.array([25]), np.array([2])], 2013:[np.array([250])],
                         2014:[np.array([400]), np.array([2]), np.array([25])], 2015:[np.array([450])]}
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*365)

        event, all_events, no_event, all_no_events = evaluate_EWRs.lowflow_check(EWR_info, iteration, flow, event, all_events, no_event, all_no_events, water_years)

        # Expected results - TEST 1: #
        expected_event = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
        expected_all_events = {2012:[np.array([10]*10), np.array([15]*12)], 2013:[np.array([10]*50)], 
                               2014:[np.array([10]*10), np.array([15]*15), np.array([10]*20)], 2015:[]}
        expected_no_event = 0
        expected_all_no_events = {2012:[np.array([25]), np.array([2])], 2013:[np.array([250])],
                                  2014:[np.array([400]), np.array([2]), np.array([25])], 2015:[np.array([450])]}
        self.assertEqual(event,expected_event)
        for year in all_events:
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
        self.assertEqual(no_event, expected_no_event)
        self.assertDictEqual(all_no_events, expected_all_no_events)
        for year in all_no_events:
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
    
    def test_ctf_check(self):
        '''Things to check on the check functions:
        - Event requirements just triggered
        - Event requirements failed
        - event requirements failed but there is some gap track remaining
        - flow failed but event recorded
        '''
        # Set up variables for all tests:
        EWR_info = {'min_flow': 0, 'max_flow': 1}
        flow = 2
        event = [0]*9
        iteration = 365+365+365+100
        all_events = {2012:[np.array([10]*10), np.array([15]*12)], 2013:[np.array([10]*50)], 
                      2014:[np.array([10]*10), np.array([15]*15), np.array([10]*20)], 2015:[]}
        no_event = 10
        all_no_events = {2012:[np.array([25]), np.array([2])], 2013:[np.array([250])],
                         2014:[np.array([400]), np.array([2]), np.array([25])], 2015:[np.array([450])]}
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*365)

        event, all_events, no_event, all_no_events = evaluate_EWRs.ctf_check(EWR_info, iteration, flow, event, all_events, no_event, all_no_events, water_years)

        # Expected results - TEST 1: #
        expected_event = []
        expected_all_events = {2012:[np.array([10]*10), np.array([15]*12)], 2013:[np.array([10]*50)],
                               2014:[np.array([10]*10), np.array([15]*15), np.array([10]*20)], 2015:[np.array([0]*9)]}
        expected_no_event = 1
        expected_all_no_events = {2012:[np.array([25]), np.array([2])], 2013:[np.array([250])],
                                  2014:[np.array([400]), np.array([2]), np.array([25])], 2015:[np.array([450]), np.array([10])]}
        self.assertEqual(event,expected_event)
        for year in all_events:
            for i, event in enumerate(all_events[year]):
                np.array_equal(event, expected_all_events[year][i])
        self.assertEqual(no_event, expected_no_event)
        for year in all_no_events:
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
    def test_level_check(self):
        '''Things to check on the check functions:
        - Event requirements just triggered
        - Event requirements failed
        - event requirements failed but there is some gap track remaining
        - flow failed but event recorded
        '''
        # Set up variables for all tests:
        EWR_info = {'min_level': 10, 'max_level':20, 'duration': 5, 'drawdown_rate':0.04}
        level = 5
        level_change = 0.04
        water_year = 2015
        event = [10]*5
        iteration = 365+365+365+100
        all_events = {2012:[np.array([10]*10), np.array([15]*12)], 2013:[np.array([10]*50)], 
                      2014:[np.array([10]*10), np.array([15]*15), np.array([10]*20)], 2015:[]}
        no_event = 15
        all_no_events = {2012:[np.array([25]), np.array([2])], 2013:[np.array([250])],
                         2014:[np.array([400]), np.array([2]), np.array([25])], 2015:[np.array([450])]}
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*365)

        event, all_events, no_event, all_no_events = evaluate_EWRs.level_check(EWR_info, iteration, level, level_change, event, all_events, no_event, all_no_events, water_years)

        # Expected results - TEST 1: #
        expected_event = []
        expected_all_events = {2012:[np.array([10]*10), np.array([15]*12)], 2013:[np.array([10]*50)], 
                               2014:[np.array([10]*10), np.array([15]*15), np.array([10]*20)], 2015:[np.array([10]*5)]}
        expected_no_event = 1
        expected_all_no_events = {2012:[np.array([25]), np.array([2])], 2013:[np.array([250])],
                                  2014:[np.array([400]), np.array([2]), np.array([25])], 2015:[np.array([450]), np.array([10])]}
        self.assertEqual(event,expected_event)
        for year in all_events:
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
        self.assertEqual(no_event, expected_no_event)
        self.assertDictEqual(all_no_events, expected_all_no_events)
        for year in all_no_events:
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
                
    def test_flow_check_sim(self):
        '''Things to check on the check functions:
        - Event requirements just triggered
        - Event requirements failed
        - event requirements failed but there is some gap track remaining
        - flow failed but event recorded
        '''
        # Set up variables for all tests:
        iteration = 370
        EWR_info1 = {'min_flow': 10, 'max_flow': 20, 'min_event': 5, 'gap_tolerance': 0}
        EWR_info2 = {'min_flow': 10, 'max_flow': 20, 'min_event': 5, 'gap_tolerance': 0}
        flow1 = 5
        flow2 = 7
        event = [10]*5
        total_event = 5
        all_events = {2012:[np.array([10]*10), np.array([15]*12)], 2013:[np.array([10]*50)], 
                      2014:[np.array([10]*10), np.array([15]*15), np.array([10]*20)], 2015:[]}
        no_event = 25
        all_no_events = {2012:[np.array([25]), np.array([2])], 2013:[np.array([250])],
                         2014:[np.array([400]), np.array([2]), np.array([25])], 2015:[np.array([450])]}
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*365)
        gap_track = 0
        event, all_events, no_event, all_no_events, gap_track, total_event = evaluate_EWRs.flow_check_sim(iteration, EWR_info1, EWR_info2, water_years, flow1, flow2, event, all_events, no_event, all_no_events, gap_track, total_event)

        # Expected results - TEST 1: #
        expected_event = []
        expected_all_events = {2012:[np.array([10]*10), np.array([15]*12)], 2013:[np.array([10]*50), np.array([10]*5)],
                               2014:[np.array([10]*10), np.array([15]*15), np.array([10]*20)], 2015:[np.array([10]*5)]}
        expected_no_event = 1
        expected_all_no_events = {2012:[np.array([25]), np.array([2])], 2013:[np.array([250]), np.array([20])],
                                  2014:[np.array([400]), np.array([2]), np.array([25])], 2015:[np.array([450])]}
        expected_gap_track = 0
        expected_total_event = 0
        self.assertEqual(gap_track, expected_gap_track)
        self.assertEqual(event,expected_event)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
        self.assertEqual(no_event, expected_no_event)
        self.assertDictEqual(all_no_events, expected_all_no_events)
        for year in all_no_events:
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
                
    def test_flow_calc(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        flows = np.array([0]*355+[10]*10 + [0]*355+[10]*10 + [0]*355+[10]*10 + [0]*355+[10]*10+[10]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [np.array([10]*10)], 2013: [np.array([10]*10)], 2014: [np.array([10]*10)], 2015: [np.array([10]*11)]}
        expected_all_no_events = {2012: [np.array([355])], 2013: [np.array([355])], 2014: [np.array([355])], 2015: [np.array([355])]}
        expected_durations = [10]*4
        all_events, all_no_events, durations = evaluate_EWRs.flow_calc(EWR_info, flows, water_years, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)
        
    def test_lowflow_calc(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'min_event':1, 'duration': 300, 'duration_VD': 10}
        flows = np.array([5]*295+[0]*25+[10]*45 + [0]*355+[5000]*10 + [0]*355+[10]*10 + [5]*295+[0]*25+[10]*45+[10]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Wet']*365 + ['Very Dry']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [np.array([5]*295), np.array([10]*45)], 2013: [], 2014: [np.array([10]*10)], 2015: [np.array([5]*295), np.array([10]*46)]}
        expected_all_no_events = {2012: [np.array([25])], 2013: [], 2014: [np.array([720])], 2015: [np.array([25])]}
        expected_durations = [300,300,10,300] # adding in a very dry year climate year
        all_events, all_no_events, durations = evaluate_EWRs.lowflow_calc(EWR_info, flows, water_years, climates, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)

    def test_ctf_calc(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info = {'min_flow': 0, 'max_flow': 1, 'min_event':5, 'duration': 20, 'duration_VD': 10}
        flows = np.array([5]*295+[0]*25+[10]*45 + [20]*355+[5000]*5+[0]*5 + [0]*355+[10]*10 + [1]*295+[20]*25+[0]*45+[0]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Wet']*365 + ['Very Dry']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [np.array([0]*25)], 2013: [np.array([0]*5)], 2014: [np.array([0]*355)], 2015: [np.array([1]*295), np.array([0]*46)]}
        expected_all_no_events = {2012: [np.array([295])], 2013: [np.array([405])], 2014: [], 2015: [np.array([10]), np.array([25])]}
        expected_durations = [20,20,10,20] # adding in a very dry year climate year
        all_events, all_no_events, durations = evaluate_EWRs.ctf_calc(EWR_info, flows, water_years, climates, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                np.array_equal(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                np.array_equal(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)

    def test_ctf_calc_anytime(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info = {'min_flow': 0, 'max_flow': 1, 'min_event':5, 'duration': 20, 'duration_VD': 10}
        flows = np.array([5]*295+[0]*25+[10]*45 + [20]*355+[5000]*5+[0]*5 + [0]*355+[10]*10 + [1]*295+[20]*25+[0]*45+[0]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Wet']*365 + ['Very Dry']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [np.array([0]*25)], 2013: [], 2014: [np.array([0]*360)], 2015: [np.array([1]*295), np.array([0]*46)]}
        expected_all_no_events = {2012: [np.array([295])], 2013: [np.array([405])], 2014: [], 2015: [np.array([10]), np.array([25])]}
        expected_durations = [20,20,10,20] # adding in a very dry year climate year
        all_events, all_no_events, durations = evaluate_EWRs.ctf_calc_anytime(EWR_info, flows, water_years, climates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                np.array_equal(event, expected_all_events[year][i])
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                np.array_equal(no_event, expected_all_no_events[year][i])
        self.assertEqual(durations, expected_durations)
        
    def test_flow_calc_anytime(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        flows = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[0]*350+[10]*10+[10]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        
        expected_all_events = {2012: [np.array([10]*10)], 2013: [], 2014: [np.array([10]*20), np.array([10]*15)], 2015: [np.array([10]*11)]}
        expected_all_no_events = {2012: [np.array([350])], 2013: [np.array([360])], 2014: [np.array([345])], 2015: [np.array([350])]}
        expected_durations = [10]*4
        all_events, all_no_events, durations = evaluate_EWRs.flow_calc_anytime(EWR_info, flows, water_years)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)
        
    def test_lake_calc(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info = {'min_level': 50, 'max_level': 60, 'duration': 10, 'min_event': 10, 'drawdown_rate': 0.04}
        levels = np.array([0]*350+[50]*10+[0]*5 + [0]*355+[61]*10 + [50]*10+[0]*345+[50]*10 + [50]*5+[0]*350+[50]*10+[50]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [np.array([50]*10)], 2013: [], 2014: [np.array([50]*10)], 2015: [np.array([50]*11)]}
        expected_all_no_events = {2012: [np.array([350])], 2013: [], 2014: [np.array([725])], 2015: [np.array([355])]}
        expected_durations = [10]*4
        all_events, all_no_events, durations = evaluate_EWRs.lake_calc(EWR_info, levels, water_years, dates, masked_dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)

    def test_cumulative_calc(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info = {'min_volume': 100, 'min_flow': 50, 'duration': 2}
        flows = np.array([0]*350+[10]*10+[20]*5 + [0]*360+[100]*5 + [75]*1+[25]*1+[0]*353+[50]*10 + [50]*2+[0]*362+[49]*1+[100]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [], 
                               2013: [np.array([100]), np.array([100]*2), np.array([100]*2)], 
                               2014: [np.array([50]*2), np.array([50]*2), np.array([50]*2), np.array([50]*2), np.array([50]*2)], 
                               2015: [np.array([50]*2), np.array([100]*1)]}
        expected_all_no_events = {2012: [], 2013: [np.array([724])], 2014: [np.array([355])], 2015: [np.array([362])]}
        expected_durations = [2]*4
        all_events, all_no_events, durations = evaluate_EWRs.cumulative_calc(EWR_info, flows, water_years, dates, masked_dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)
        
        
    def test_cumulative_calc_anytime(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info = {'min_volume': 100, 'min_flow': 50, 'duration': 2}
        flows = np.array([0]*350+[10]*14+[50]*1 + [50]*1+[0]*358+[100]*6 + [75]*1+[25]*1+[0]*353+[50]*10 + [50]*2+[0]*362+[49]*1+[100]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        
        expected_all_events = {2012: [], 
                               2013: [np.array([50]*2), np.array([100]*1), np.array([100]*2), np.array([100]*2)], 
                               2014: [np.array([100,75]), np.array([50]*2), np.array([50]*2), np.array([50]*2), np.array([50]*2), np.array([50]*2)], 
                               2015: [np.array([50]*2), np.array([100]*1)]}
        expected_all_no_events = {2012: [], 2013: [np.array([364]), np.array([357])], 2014: [np.array([354])], 2015: [np.array([362])]}
        expected_durations = [2]*4
        all_events, all_no_events, durations = evaluate_EWRs.cumulative_calc_anytime(EWR_info, flows, water_years)       
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)

        
    def test_nest_calc_weirpool(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'drawdown_rate': 0.04, 'duration': 10}
        flows = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[0]*351+[10]*10)
        levels = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*9+[1]*1 + [10]*5+[0]*351+[10]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [np.array([10]*10)], 2013: [np.array([10]*10)], 2014: [np.array([10]*10)], 2015: [np.array([10]*10)]}
        expected_all_no_events = {2012: [np.array([350])], 2013: [np.array([360])], 2014: [], 2015: [np.array([711])]}
        expected_durations = [10]*4
        all_events, all_no_events, durations = evaluate_EWRs.nest_calc_weirpool(EWR_info, flows, levels, water_years, dates, masked_dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)
        
    def test_nest_calc_percent(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'drawdown_rate': '10%', 'duration': 10}
        flows = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*9+[8]*1 + [10]*9+[9]*1+[0]*346+[10]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [np.array([10]*10)], 2013: [np.array([10]*10)], 2014: [np.array([10]*10)], 2015: [np.array([10]*9+[9]*1), np.array([10]*10)]}
        expected_all_no_events = {2012: [np.array([350])], 2013: [np.array([360])], 2014: [], 2015: [np.array([355]), np.array([346])]}
        expected_durations = [10]*4
        all_events, all_no_events, durations = evaluate_EWRs.nest_calc_percent(EWR_info, flows, water_years, dates, masked_dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)
        
    def test_nest_calc_percent_trigger(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'drawdown_rate': '10%', 'duration': 10, 'trigger_day': 15, 'trigger_month': 10}
        flows = np.array([0]*106+[11]*1+[10]*9+[0]*249 + [0]*106+[9]*1+[10]*9+[0]*249 + [0]*106+[10]*9+[9]*1+[0]*249 + [0]*106+[10]*9+[8]*1+[0]*250)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [np.array([11]*1+[10]*9)], 2013: [np.array([9]*1+[10]*9)], 2014: [np.array([10]*9+[9]*1)], 2015: []}
        expected_all_no_events = {2012: [np.array([106])], 2013: [np.array([355])], 2014: [np.array([355])], 2015: [np.array([615])]}
        expected_durations = [10]*4
        all_events, all_no_events, durations = evaluate_EWRs.nest_calc_percent_trigger(EWR_info, flows, water_years, dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)

        
    def test_weirpool_calc(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Testing weirpool falling #
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'min_level': 5, 'max_level': 10, 'drawdown_rate': 0.04, 'duration': 10}
        flows = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[8]*5+[0]*346+[10]*10)
        levels = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*9+[1]*1 + [11]*5+[10]*5+[0]*346+[11]*1+[10]*9)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [np.array([10]*10)], 2013: [np.array([10]*10)], 2014: [np.array([10]*10)], 2015: []}
        expected_all_no_events = {2012: [np.array([350])], 2013: [np.array([360])], 2014: [], 2015: [np.array([721])]}
        expected_durations = [10]*4
        weirpool_type = 'falling'
        all_events, all_no_events, durations = evaluate_EWRs.weirpool_calc(EWR_info, flows, levels, water_years, weirpool_type, dates, masked_dates)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)
        
        # Testing weirpool raising #
        EWR_info = {'min_flow': 5, 'max_flow': 20, 'min_level': 10, 'max_level': 20, 'drawdown_rate': 0.04, 'duration': 10}
        flows = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[8]*5+[0]*346+[10]*10)
        levels = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*9+[1]*1 + [11]*5+[10.96]*5+[0]*346+[10]*9+[9.96]*1)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [np.array([10]*10)], 2013: [np.array([10]*10)], 2014: [np.array([10]*10)], 2015: [np.array([10]*5+[8]*5)]}
        expected_all_no_events = {2012: [np.array([350])], 2013: [np.array([360])], 2014: [], 2015: [np.array([355]), np.array([356])]}
        expected_durations = [10]*4
        weirpool_type = 'raising'
        all_events, all_no_events, durations = evaluate_EWRs.weirpool_calc(EWR_info, flows, levels, water_years, weirpool_type, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)   

        
    def test_flow_calc_anytime_sim(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info1 = {'min_flow': 10, 'max_flow': 20, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        EWR_info2 = {'min_flow': 20, 'max_flow': 30, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        
        flows1 = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[0]*351+[10]*10)
        flows2 = np.array([0]*350+[30]*10+[0]*5 + [0]*355+[30]*10 + [30]*10+[0]*345+[10]*10 + [10]*10+[0]*346+[30]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        
        
        expected_all_events = {2012: [np.array([10]*10)], 2013: [], 2014: [np.array([10]*20)], 2015: [np.array([10]*10)]}
        expected_all_no_events = {2012: [np.array([350])], 2013: [np.array([360])], 2014: [], 2015: [np.array([711])]}
        expected_durations = [10]*4
        all_events, all_no_events, durations = evaluate_EWRs.flow_calc_anytime_sim(EWR_info1, EWR_info2, flows1, flows2, water_years)
#         print(all_events)
#         print(expected_all_events)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
#         print(all_no_events)
#         print(expected_all_no_events)
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)

        
    def test_flow_calc_sim(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info1 = {'min_flow': 10, 'max_flow': 20, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        EWR_info2 = {'min_flow': 20, 'max_flow': 30, 'gap_tolerance': 0, 'min_event':10, 'duration': 10}
        
        flows1 = np.array([0]*350+[10]*10+[0]*5 + [0]*355+[10]*10 + [10]*10+[0]*345+[10]*10 + [10]*5+[0]*351+[10]*10)
        flows2 = np.array([0]*350+[30]*10+[0]*5 + [0]*355+[30]*10 + [30]*10+[0]*345+[10]*10 + [10]*10+[0]*346+[30]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events = {2012: [np.array([10]*10)], 2013: [np.array([10]*10)], 2014: [np.array([10]*10)], 2015: [np.array([10]*10)]}
        expected_all_no_events = {2012: [np.array([350])], 2013: [np.array([360])], 2014: [], 2015: [np.array([711])]}
        expected_durations = [10]*4
        all_events, all_no_events, durations = evaluate_EWRs.flow_calc_sim(EWR_info1, EWR_info2, flows1, flows2, water_years, dates, masked_dates)
        for year in all_events:
            self.assertEqual(len(all_events[year]), len(expected_all_events[year]))
            for i, event in enumerate(all_events[year]):
                self.assertTrue(np.array_equal(event, expected_all_events[year][i]))
        for year in all_no_events:
            self.assertEqual(len(all_no_events[year]), len(expected_all_no_events[year]))
            for i, no_event in enumerate(all_no_events[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events[year][i]))
        self.assertEqual(durations, expected_durations)
        
    def test_lowflow_calc_sim(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info1 = {'min_flow': 10, 'max_flow': 20, 'duration': 10, 'duration_VD': 5}
        EWR_info2 = {'min_flow': 20, 'max_flow': 30, 'duration': 10, 'duration_VD': 5}
        
        flows1 = np.array([10]*1+[0]*350+[10]*9+[0]*5 + [0]*360+[10]*5 + [10]*10+[0]*345+[10]*10 + [8]*5+[0]*351+[10]*10)
        flows2 = np.array([25]*1+[0]*350+[30]*9+[0]*5 + [0]*360+[30]*5 + [30]*10+[0]*345+[10]*10 + [18]*10+[0]*346+[30]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Dry']*365 +['Very Wet']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events1 = {2012: [np.array([10]*1), np.array([10]*9)], 2013: [np.array([10]*5)], 2014: [np.array([10]*10), np.array([10]*10)], 2015: [np.array([10]*10)]}
        expected_all_events2 = {2012: [np.array([25]*1), np.array([30]*9)], 2013: [np.array([30]*5)], 2014: [np.array([30]*10)], 2015: [np.array([30]*10)]}
        
        expected_all_no_events1 = {2012: [np.array([350])], 2013: [np.array([365])], 2014: [np.array([345])], 2015: [np.array([356])]}
        expected_all_no_events2 = {2012: [np.array([350])], 2013: [np.array([365])], 2014: [], 2015: [np.array([711])]}
        
        expected_durations = [10,5,10,10]
        all_events1, all_events2, all_no_events1, all_no_events2, durations = evaluate_EWRs.lowflow_calc_sim(EWR_info1, EWR_info2, flows1, flows2, water_years, climates, dates, masked_dates)
#         print(all_events1)
#         print(expected_all_events1)
        for year in all_events1:
            self.assertEqual(len(all_events1[year]), len(expected_all_events1[year]))
            for i, event in enumerate(all_events1[year]):
                self.assertTrue(np.array_equal(event, expected_all_events1[year][i]))
#         print(all_events2)
#         print(expected_all_events2)
        for year in all_events2:
            self.assertEqual(len(all_events2[year]), len(expected_all_events2[year]))
            for i, event in enumerate(all_events2[year]):
                self.assertTrue(np.array_equal(event, expected_all_events2[year][i]))
#         print(all_no_events1)
#         print(expected_all_no_events1)
        for year in all_no_events1:
            self.assertEqual(len(all_no_events1[year]), len(expected_all_no_events1[year]))
            for i, no_event in enumerate(all_no_events1[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events1[year][i]))
#         print(all_no_events2)
#         print(expected_all_no_events2)
        for year in all_no_events2:
            self.assertEqual(len(all_no_events2[year]), len(expected_all_no_events2[year]))
            for i, no_event in enumerate(all_no_events2[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events2[year][i]))
        self.assertEqual(durations, expected_durations)        
        
        
    def test_ctf_calc_sim(self):
        '''Things to check on this function:
        - event at start of year
        - event at end of year
        - multiple events '''
        # Setting up data for all tests #
        EWR_info1 = {'min_flow': 0, 'max_flow': 1, 'duration': 10, 'duration_VD': 5}
        EWR_info2 = {'min_flow': 0, 'max_flow': 1, 'duration': 10, 'duration_VD': 5}
        
        flows1 = np.array([10]*1+[0]*350+[10]*9+[1]*5 + [0]*360+[10]*5 + [10]*10+[0]*345+[10]*1+[1]*9 + [8]*5+[10]*351+[0]*10)
        flows2 = np.array([10]*1+[0]*350+[10]*9+[1]*5 + [0]*360+[10]*5 + [10]*10+[0]*345+[10]*1+[1]*9 + [8]*5+[10]*351+[0]*10)
        water_years = np.array([2012]*365 + [2013]*365 + [2014]*365 + [2015]*366)
        climates = np.array(['Wet']*365 + ['Very Dry']*365 +['Very Wet']*365 + ['Dry']*366)
        dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        masked_dates = pd.date_range(start= datetime.strptime('2012-07-01', '%Y-%m-%d'), end = datetime.strptime('2016-06-30', '%Y-%m-%d'))
        
        
        expected_all_events1 = {2012: [np.array([0]*350), np.array([1]*5)], 2013: [np.array([0]*360)], 2014: [np.array([0]*345), np.array([1]*9)], 2015: [np.array([0]*10)]}
        expected_all_events2 = {2012: [np.array([0]*350), np.array([1]*5)], 2013: [np.array([0]*360)], 2014: [np.array([0]*345), np.array([1]*9)], 2015: [np.array([0]*10)]}
        
        expected_all_no_events1 = {2012: [np.array([1]), np.array([9])], 2013: [], 2014: [np.array([15]), np.array([1])], 2015: [np.array([356])]}
        expected_all_no_events2 = {2012: [np.array([1]), np.array([9])], 2013: [], 2014: [np.array([15]), np.array([1])], 2015: [np.array([356])]}
        
        expected_durations = [10,5,10,10]
        all_events1, all_events2, all_no_events1, all_no_events2, durations = evaluate_EWRs.ctf_calc_sim(EWR_info1, EWR_info2, flows1, flows2, water_years, climates, dates, masked_dates)
#         print(all_events1)
#         print(expected_all_events1)
        for year in all_events1:
            self.assertEqual(len(all_events1[year]), len(expected_all_events1[year]))
            for i, event in enumerate(all_events1[year]):
                self.assertTrue(np.array_equal(event, expected_all_events1[year][i]))
#         print(all_events2)
#         print(expected_all_events2)
        for year in all_events2:
            self.assertEqual(len(all_events2[year]), len(expected_all_events2[year]))
            for i, event in enumerate(all_events2[year]):
                self.assertTrue(np.array_equal(event, expected_all_events2[year][i]))
#         print(all_no_events1)
#         print(expected_all_no_events1)
        for year in all_no_events1:
            self.assertEqual(len(all_no_events1[year]), len(expected_all_no_events1[year]))
            for i, no_event in enumerate(all_no_events1[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events1[year][i]))
#         print(all_no_events2)
#         print(expected_all_no_events2)
        for year in all_no_events2:
            self.assertEqual(len(all_no_events2[year]), len(expected_all_no_events2[year]))
            for i, no_event in enumerate(all_no_events2[year]):
                self.assertTrue(np.array_equal(no_event, expected_all_no_events2[year][i]))
        self.assertEqual(durations, expected_durations)

In [187]:
Test = test_evaluate_EWRs()
suite = unittest.TestSuite()

suite.addTest(test_evaluate_EWRs('test_component_pull'))
suite.addTest(test_evaluate_EWRs('test_apply_correction'))
suite.addTest(test_evaluate_EWRs('test_get_EWRs'))
suite.addTest(test_evaluate_EWRs('test_mask_dates'))
suite.addTest(test_evaluate_EWRs('test_wateryear_daily'))
suite.addTest(test_evaluate_EWRs('test_which_wateryear'))
suite.addTest(test_evaluate_EWRs('test_get_duration'))
suite.addTest(test_evaluate_EWRs('test_construct_event_dict'))
suite.addTest(test_evaluate_EWRs('test_get_days_between'))
suite.addTest(test_evaluate_EWRs('test_get_event_years'))
suite.addTest(test_evaluate_EWRs('test_get_achievements'))
suite.addTest(test_evaluate_EWRs('test_get_number_events'))
suite.addTest(test_evaluate_EWRs('test_get_average_event_length'))
suite.addTest(test_evaluate_EWRs('test_get_total_days'))
suite.addTest(test_evaluate_EWRs('test_get_data_gap'))
suite.addTest(test_evaluate_EWRs('test_check_flow'))
suite.addTest(test_evaluate_EWRs('test_lowflow_check'))
suite.addTest(test_evaluate_EWRs('test_ctf_check'))
suite.addTest(test_evaluate_EWRs('test_level_check'))
suite.addTest(test_evaluate_EWRs('test_flow_check_sim'))
suite.addTest(test_evaluate_EWRs('test_flow_calc'))
suite.addTest(test_evaluate_EWRs('test_lowflow_calc'))
suite.addTest(test_evaluate_EWRs('test_ctf_calc'))
suite.addTest(test_evaluate_EWRs('test_ctf_calc_anytime'))
suite.addTest(test_evaluate_EWRs('test_flow_calc_anytime'))
suite.addTest(test_evaluate_EWRs('test_lake_calc'))
suite.addTest(test_evaluate_EWRs('test_cumulative_calc'))
suite.addTest(test_evaluate_EWRs('test_cumulative_calc_anytime'))
suite.addTest(test_evaluate_EWRs('test_nest_calc_weirpool'))
suite.addTest(test_evaluate_EWRs('test_nest_calc_percent'))
suite.addTest(test_evaluate_EWRs('test_nest_calc_percent_trigger'))
suite.addTest(test_evaluate_EWRs('test_weirpool_calc'))
suite.addTest(test_evaluate_EWRs('test_flow_calc_anytime_sim'))
suite.addTest(test_evaluate_EWRs('test_flow_calc_sim'))
suite.addTest(test_evaluate_EWRs('test_lowflow_calc_sim'))
suite.addTest(test_evaluate_EWRs('test_ctf_calc_sim'))


unittest.TextTestRunner().run(suite)

....................................
----------------------------------------------------------------------
Ran 36 tests in 1.865s

OK


<unittest.runner.TextTestResult run=36 errors=0 failures=0>

In [159]:

type(test_set)

set

In [189]:
400*26

10400

In [35]:
class test_tests(unittest.TestCase):
    def test_sets(self):
        test = [4,4,4,5,6,7,8,9]
        test_set = set(test)
        self.assertEqual(test_set, test_set)

In [36]:
Test = test_tests()
suite = unittest.TestSuite()

suite.addTest(test_tests('test_sets'))

unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>